# Extraction of the "Signaling by EGFR" pathway (R-HSA-177929) from the BioPAX export (v65) of Reactome

#### Import libraries

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import IPython
import pandas as pd
import subprocess
import time

#### Define URL of SPARQL endpoint

In [2]:
endpointURL_reactome_v65 = "http://localhost:3030/reactome_v65/query"
rdfFormat = "turtle"

#### Define RDF prefixes

In [3]:
reactomeVersion = 65 
prefixes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>

PREFIX chebi: <http://purl.obolibrary.org/obo/chebi/>
PREFIX chebidb: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX chebirel: <http://purl.obolibrary.org/obo/CHEBI#>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

PREFIX bp3: <http://www.biopax.org/release/biopax-level3.owl#>

# Homo_sapiens-20170221.owl
#PREFIX reactome: <http://www.reactome.org/biopax/59/48887#> 
#
# Homo_sapiens-20210608.owl
#PREFIX reactome: <http://www.reactome.org/biopax/77/48887#>
#
# Homo_sapiens-20220614.owl
#PREFIX reactome: <http://www.reactome.org/biopax/81/48887#>
#
# Homo_sapiens-20221130.owl
#PREFIX reactome: <http://www.reactome.org/biopax/83/48887#>

PREFIX reactome: <http://www.reactome.org/biopax/{}/48887#>
""".format(reactomeVersion)

biopaxURI = "http://www.biopax.org/release/biopax-level3.owl#"

# Useful functions

In [4]:
def displaySparqlResults(results):
    '''
    Displays as HTML the result of a SPARQLWrapper query in a Jupyter notebook.
    
        Parameters:
            results (dictionnary): the result of a call to SPARQLWrapper.query().convert()
    '''
    variableNames = results['head']['vars']
    tableCode = '<table><tr><th>{}</th></tr><tr>{}</tr></table>'.format('</th><th>'.join(variableNames), '</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join([row[vName]['value'] if vName in row.keys() else "&nbsp;" for vName in variableNames]))for row in results["results"]["bindings"]))
    IPython.display.display(IPython.display.HTML(tableCode))

In [5]:
def getPathwayURIbyName(pathwayName):
    query="""
SELECT DISTINCT ?pathwayURI ?pathwayReactomeID
WHERE {
  ?pathwayURI bp3:name|bp3:displayName \"""" + pathwayName + """\" .
  ?pathwayURI bp3:xref [ rdf:type bp3:UnificationXref ;
                      bp3:db "Reactome" ;
                      bp3:id ?pathwayReactomeID ] .
}
"""
    sparql = SPARQLWrapper(endpointURL_reactome_v65)
    sparql.setQuery(prefixes+query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results

# Launch SPARQL endpoint loading the standalone BioPAX export of Reactome (version 65)

In [6]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/ReactomeBioPAX/_00_Reactome_Data_v65/Homo_sapiens.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/reactome_v65']
process = subprocess.Popen(command)
time.sleep(60)

15:22:51 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/ReactomeBioPAX/_00_Reactome_Data_v65/Homo_sapiens.owl
15:22:52 WARN  riot            :: [line: 67280, col: 52] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
15:23:12 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
15:23:12 INFO  Server          :: Running in read-only mode for /reactome_v65
15:23:12 INFO  Server          :: Apache Jena Fuseki 4.9.0
15:23:12 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
15:23:12 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figures6-7/Scripts/run
15:23:12 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figures6-7/Scripts/run/shiro.ini
15:23:12 INFO  Server 

# Extraction of "Signaling by EGFR" (R-HSA-177929) pathway from Reactome BioPAX export v65

## 1 - Extract direct pathway components

In [7]:
defaultPrefixValue = "http://www.reactome.org/biopax/{}/48887#".format(reactomeVersion)
pathwayIdent = "http://www.reactome.org/biopax/65/48887#Pathway2275"
if pathwayIdent.startswith(defaultPrefixValue):
        pathwayIdent = pathwayIdent.replace(defaultPrefixValue, "reactome" + ':')
if not(pathwayIdent.startswith("reactome" + ':')):
        pathwayIdent = "<" + pathwayIdent + ">"
query = """
SELECT DISTINCT ?pathwayStart ?pathway_component ?direct_component
WHERE {
  VALUES ?pathwayStart { reactome:Pathway2275 } # "Signaling by EGFR" in Reactome BioPAX v65 (Pathway165 in Reactome BioPAX v38)
  
  # Pathway components
  ?pathwayStart bp3:pathwayComponent ?direct_component .
  ?pathwayStart ?pathway_component ?direct_component .
}
"""
sparql = SPARQLWrapper(endpointURL_reactome_v65)
sparql.setQuery(prefixes+query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/1_direct_pathway_compo.csv", "wb") as f:
    f.write(results)

15:23:51 INFO  Fuseki          :: [1] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.rea

pathwayStart,pathway_component,direct_component
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12383
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12384
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#Pathway2277
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12385
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#Pathway2276
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12381
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12386
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#Pathway2280
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12382
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12387


15:23:51 INFO  Fuseki          :: [2] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.rea

## 2 - Description of direct pathway components of type BiochemicalReaction

In [8]:
defaultPrefixValue = "http://www.reactome.org/biopax/{}/48887#".format(reactomeVersion)
pathwayIdent = "http://www.reactome.org/biopax/65/48887#Pathway2275"
if pathwayIdent.startswith(defaultPrefixValue):
        pathwayIdent = pathwayIdent.replace(defaultPrefixValue, "reactome" + ':')
if not(pathwayIdent.startswith("reactome" + ':')):
        pathwayIdent = "<" + pathwayIdent + ">"
query = """
SELECT DISTINCT ?direct_component ?relations_pathway_compo_br ?linked_pathway_compo_br
WHERE {
  VALUES ?pathwayStart { reactome:Pathway2275 } # "Signaling by EGFR" in Reactome BioPAX v65 (Pathway165 in Reactome BioPAX v38)
  
  # Pathway components
  ?pathwayStart bp3:pathwayComponent ?direct_component .
  ?pathwayStart ?pathway_component ?direct_component .
  ?direct_component rdf:type bp3:BiochemicalReaction .
  VALUES ?relations_pathway_compo_br { bp3:displayName bp3:left bp3:right bp3:conversionDirection bp3:participantStoichiometry}
  ?direct_component ?relations_pathway_compo_br ?linked_pathway_compo_br .
}
"""
sparql = SPARQLWrapper(endpointURL_reactome_v65)
sparql.setQuery(prefixes+query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/2_direct_pathway_compo_br.csv", "wb") as f:
    f.write(results)

15:23:51 INFO  Fuseki          :: [3] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.rea

direct_component,relations_pathway_compo_br,linked_pathway_compo_br
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12383,http://www.biopax.org/release/biopax-level3.owl#displayName,"FAM83B, (FAM83A, FAM83D) bind EGFR"
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12384,http://www.biopax.org/release/biopax-level3.owl#displayName,EGFR binds EGF ligand
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12385,http://www.biopax.org/release/biopax-level3.owl#displayName,EGFR dimerization
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12381,http://www.biopax.org/release/biopax-level3.owl#displayName,Pro-EGF is cleaved to form mature EGF
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12386,http://www.biopax.org/release/biopax-level3.owl#displayName,EGFR autophosphorylation
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12382,http://www.biopax.org/release/biopax-level3.owl#displayName,AAMP binds EGFR
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12387,http://www.biopax.org/release/biopax-level3.owl#displayName,Phosphorylation of EGFR by SRC kinase
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12383,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Protein29336
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12383,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Protein8391
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12384,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Protein8392


15:23:51 INFO  Fuseki          :: [4] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.rea

## 3 - Description of direct pathway components of type Pathway

In [9]:
defaultPrefixValue = "http://www.reactome.org/biopax/{}/48887#".format(reactomeVersion)
pathwayIdent = "http://www.reactome.org/biopax/65/48887#Pathway2275"
if pathwayIdent.startswith(defaultPrefixValue):
        pathwayIdent = pathwayIdent.replace(defaultPrefixValue, "reactome" + ':')
if not(pathwayIdent.startswith("reactome" + ':')):
        pathwayIdent = "<" + pathwayIdent + ">"
query = """
SELECT DISTINCT ?direct_component ?relations_pathway_compo_path ?linked_pathway_compo_path
WHERE {
  VALUES ?pathwayStart { reactome:Pathway2275 } # "Signaling by EGFR" in Reactome BioPAX v65 (Pathway165 in Reactome BioPAX v38)
  
  # Pathway components
  ?pathwayStart bp3:pathwayComponent ?direct_component .
  ?pathwayStart ?pathway_component ?direct_component .
  ?direct_component rdf:type bp3:Pathway .
  VALUES ?relations_pathway_compo_path { bp3:displayName bp3:pathwayComponent bp3:pathwayOrder }
  ?direct_component ?relations_pathway_compo_path ?linked_pathway_compo_path .
}
"""
sparql = SPARQLWrapper(endpointURL_reactome_v65)
sparql.setQuery(prefixes+query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/3_direct_pathway_compo_path.csv", "wb") as f:
    f.write(results)

15:23:51 INFO  Fuseki          :: [5] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.rea

direct_component,relations_pathway_compo_path,linked_pathway_compo_path
http://www.reactome.org/biopax/65/48887#Pathway2277,http://www.biopax.org/release/biopax-level3.owl#displayName,GRB2 events in EGFR signaling
http://www.reactome.org/biopax/65/48887#Pathway2276,http://www.biopax.org/release/biopax-level3.owl#displayName,EGFR interacts with phospholipase C-gamma
http://www.reactome.org/biopax/65/48887#Pathway2280,http://www.biopax.org/release/biopax-level3.owl#displayName,EGFR downregulation
http://www.reactome.org/biopax/65/48887#Pathway2279,http://www.biopax.org/release/biopax-level3.owl#displayName,GAB1 signalosome
http://www.reactome.org/biopax/65/48887#Pathway2278,http://www.biopax.org/release/biopax-level3.owl#displayName,SHC1 events in EGFR signaling
http://www.reactome.org/biopax/65/48887#Pathway2277,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12392
http://www.reactome.org/biopax/65/48887#Pathway2277,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12393
http://www.reactome.org/biopax/65/48887#Pathway2277,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12391
http://www.reactome.org/biopax/65/48887#Pathway2276,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12388
http://www.reactome.org/biopax/65/48887#Pathway2276,http://www.biopax.org/release/biopax-level3.owl#pathwayComponent,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12389


15:23:51 INFO  Fuseki          :: [6] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.rea

## 4 - Extraction of direct pathway steps

In [10]:
defaultPrefixValue = "http://www.reactome.org/biopax/{}/48887#".format(reactomeVersion)
pathwayIdent = "http://www.reactome.org/biopax/65/48887#Pathway2275"
if pathwayIdent.startswith(defaultPrefixValue):
        pathwayIdent = pathwayIdent.replace(defaultPrefixValue, "reactome" + ':')
if not(pathwayIdent.startswith("reactome" + ':')):
        pathwayIdent = "<" + pathwayIdent + ">"
query = """
SELECT DISTINCT ?pathwayStart ?pathway_order ?direct_ps
WHERE {
  VALUES ?pathwayStart { reactome:Pathway2275 } # "Signaling by EGFR" in Reactome BioPAX v65 (Pathway165 in Reactome BioPAX v38)
  
  # DIRECT PATHWAY STEPS
  ?pathwayStart bp3:pathwayOrder ?direct_ps .
  ?pathwayStart ?pathway_order ?direct_ps .
  ?direct_ps rdf:type bp3:PathwayStep .
}
"""
sparql = SPARQLWrapper(endpointURL_reactome_v65)
sparql.setQuery(prefixes+query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/4_direct_pathway_steps.csv", "wb") as f:
    f.write(results)

15:23:51 INFO  Fuseki          :: [7] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.rea

pathwayStart,pathway_order,direct_ps
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayOrder,http://www.reactome.org/biopax/65/48887#PathwayStep14660
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayOrder,http://www.reactome.org/biopax/65/48887#PathwayStep14651
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayOrder,http://www.reactome.org/biopax/65/48887#PathwayStep14664
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayOrder,http://www.reactome.org/biopax/65/48887#PathwayStep14655
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayOrder,http://www.reactome.org/biopax/65/48887#PathwayStep14652
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayOrder,http://www.reactome.org/biopax/65/48887#PathwayStep14704
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayOrder,http://www.reactome.org/biopax/65/48887#PathwayStep14653
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayOrder,http://www.reactome.org/biopax/65/48887#PathwayStep14656
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayOrder,http://www.reactome.org/biopax/65/48887#PathwayStep14650
http://www.reactome.org/biopax/65/48887#Pathway2275,http://www.biopax.org/release/biopax-level3.owl#pathwayOrder,http://www.reactome.org/biopax/65/48887#PathwayStep14669


15:23:51 INFO  Fuseki          :: [8] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.rea

## 5 - Description of direct pathway steps 

In [11]:
defaultPrefixValue = "http://www.reactome.org/biopax/{}/48887#".format(reactomeVersion)
pathwayIdent = "http://www.reactome.org/biopax/65/48887#Pathway2275"
if pathwayIdent.startswith(defaultPrefixValue):
        pathwayIdent = pathwayIdent.replace(defaultPrefixValue, "reactome" + ':')
if not(pathwayIdent.startswith("reactome" + ':')):
        pathwayIdent = "<" + pathwayIdent + ">"
query = """
SELECT DISTINCT ?direct_ps ?relations_of_interest ?linked_to_ps
WHERE {
  VALUES ?pathwayStart { reactome:Pathway2275 } # "Signaling by EGFR" in Reactome BioPAX v65 (Pathway165 in Reactome BioPAX v38)
  
  # DIRECT PATHWAY STEPS
  ?pathwayStart bp3:pathwayOrder ?direct_ps .
  ?pathwayStart ?pathway_order ?direct_ps .
  ?direct_ps rdf:type bp3:PathwayStep .
  
  # RELATIONS OF PATHWAY STEPS
  VALUES ?relations_of_interest { bp3:stepProcess bp3:nextStep }
  ?direct_ps ?relations_of_interest ?linked_to_ps .
}
"""
sparql = SPARQLWrapper(endpointURL_reactome_v65)
sparql.setQuery(prefixes+query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/5_direct_pathway_steps_next_steps_and_step_process.csv", "wb") as f:
    f.write(results)

15:23:51 INFO  Fuseki          :: [9] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.rea

direct_ps,relations_of_interest,linked_to_ps
http://www.reactome.org/biopax/65/48887#PathwayStep14660,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#Pathway2276
http://www.reactome.org/biopax/65/48887#PathwayStep14651,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12382
http://www.reactome.org/biopax/65/48887#PathwayStep14664,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#Pathway2277
http://www.reactome.org/biopax/65/48887#PathwayStep14655,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12386
http://www.reactome.org/biopax/65/48887#PathwayStep14655,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#Control2248
http://www.reactome.org/biopax/65/48887#PathwayStep14655,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#Catalysis5433
http://www.reactome.org/biopax/65/48887#PathwayStep14652,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12383
http://www.reactome.org/biopax/65/48887#PathwayStep14704,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#Pathway2280
http://www.reactome.org/biopax/65/48887#PathwayStep14653,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12384
http://www.reactome.org/biopax/65/48887#PathwayStep14653,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#Control2246


15:23:51 INFO  Fuseki          :: [10] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.re

## 6 - Description of step processes linked to pathway steps

In [12]:
defaultPrefixValue = "http://www.reactome.org/biopax/{}/48887#".format(reactomeVersion)
pathwayIdent = "http://www.reactome.org/biopax/65/48887#Pathway2275"
if pathwayIdent.startswith(defaultPrefixValue):
        pathwayIdent = pathwayIdent.replace(defaultPrefixValue, "reactome" + ':')
if not(pathwayIdent.startswith("reactome" + ':')):
        pathwayIdent = "<" + pathwayIdent + ">"
query = """
SELECT DISTINCT ?linked_to_ps ?relations_of_interest_starting_from_step_process ?step_process_relations
WHERE {
  VALUES ?pathwayStart { reactome:Pathway2275 } # "Signaling by EGFR" in Reactome BioPAX v65 (Pathway165 in Reactome BioPAX v38)
  
  # DIRECT PATHWAY STEPS
  ?pathwayStart bp3:pathwayOrder ?direct_ps .
  ?pathwayStart ?pathway_order ?direct_ps .
  ?direct_ps rdf:type bp3:PathwayStep .
  
  # RELATIONS OF PATHWAY STEPS
  VALUES ?relations_of_interest { bp3:stepProcess bp3:nextStep }
  ?direct_ps ?relations_of_interest ?linked_to_ps .
  VALUES ?relations_of_interest_starting_from_step_process { bp3:left bp3:pathwayOrder bp3:pathwayComponent bp3:right bp3:controller bp3:controlled bp3:displayName bp3:conversionDirection bp3:participantStoichiometry }
  ?linked_to_ps ?relations_of_interest_starting_from_step_process ?step_process_relations .
}
"""
sparql = SPARQLWrapper(endpointURL_reactome_v65)
sparql.setQuery(prefixes+query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/6_pathway_steps_steps_processes.csv", "wb") as f:
    f.write(results)

15:23:51 INFO  Fuseki          :: [11] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.re

linked_to_ps,relations_of_interest_starting_from_step_process,step_process_relations
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12382,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Protein1554
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12382,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Protein8391
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12386,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Complex4058
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12386,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#SmallMolecule3
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12383,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Protein29336
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12383,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Protein8391
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12384,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Protein8392
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12384,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Protein8391
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12387,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Complex4058
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12387,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#SmallMolecule3


15:23:51 INFO  Fuseki          :: [12] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.re

## 7 - Description of biochemical reactions linked to direct subpathways

In [13]:
defaultPrefixValue = "http://www.reactome.org/biopax/{}/48887#".format(reactomeVersion)
pathwayIdent = "http://www.reactome.org/biopax/65/48887#Pathway2275"
if pathwayIdent.startswith(defaultPrefixValue):
        pathwayIdent = pathwayIdent.replace(defaultPrefixValue, "reactome" + ':')
if not(pathwayIdent.startswith("reactome" + ':')):
        pathwayIdent = "<" + pathwayIdent + ">"
query = """
SELECT DISTINCT ?linked_to_direct_compo ?relations_br ?description_br
WHERE {
  VALUES ?pathwayStart { reactome:Pathway2275 } # "Signaling by EGFR" in Reactome BioPAX v65 (Pathway165 in Reactome BioPAX v38)
  VALUES ?pathway_compo_types { bp3:Pathway bp3:BiochemicalReaction }
  
  # DIRECT COMPONENTS
  ?pathwayStart bp3:pathwayComponent ?direct_component .
  ?pathwayStart ?pathway_component ?direct_component .
  ?direct_component rdf:type ?pathway_compo_types .
  
  # RELATIONS OF PATHWAY COMPONENTS
  VALUES ?relations_of_interest { bp3:left bp3:right bp3:displayName bp3:conversionDirection bp3:pathwayOrder bp3:pathwayComponent bp3:participantStoichiometry }
  ?direct_component ?relations_of_interest ?linked_to_direct_compo .
  
  ?linked_to_direct_compo rdf:type bp3:BiochemicalReaction .
  VALUES ?relations_br {bp3:left bp3:right bp3:displayName bp3:conversionDirection bp3:bp3:participantStoichiometry }
  ?linked_to_direct_compo ?relations_br ?description_br
  
}
"""
sparql = SPARQLWrapper(endpointURL_reactome_v65)
sparql.setQuery(prefixes+query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/7_linked_to_br_of_direct_pathway.csv", "wb") as f:
    f.write(results)

15:23:51 INFO  Fuseki          :: [13] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.re

linked_to_direct_compo,relations_br,description_br
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12392,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Complex4459
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12392,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Complex2907
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12393,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#SmallMolecule157
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12393,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Complex306
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12391,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Protein173
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12391,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Protein1642
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12388,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Complex4459
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12388,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Protein2131
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12389,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#SmallMolecule3
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12389,http://www.biopax.org/release/biopax-level3.owl#left,http://www.reactome.org/biopax/65/48887#Complex13374


15:23:51 INFO  Fuseki          :: [14] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.re

## 8 - Description of step processes of pathway steps linked to subpathways

In [14]:
defaultPrefixValue = "http://www.reactome.org/biopax/{}/48887#".format(reactomeVersion)
pathwayIdent = "http://www.reactome.org/biopax/65/48887#Pathway2275"
if pathwayIdent.startswith(defaultPrefixValue):
        pathwayIdent = pathwayIdent.replace(defaultPrefixValue, "reactome" + ':')
if not(pathwayIdent.startswith("reactome" + ':')):
        pathwayIdent = "<" + pathwayIdent + ">"
query = """
SELECT DISTINCT ?pathwayStep ?relation ?stepProcess
WHERE {
  VALUES ?pathwayRoot { """ + pathwayIdent + """ } # Signaling by EGFR

  ?pathwayRoot bp3:pathwayComponent ?pathwayCompo .
  ?pathwayCompo bp3:pathwayOrder ?pathwayStep .
  ?pathwayStep bp3:stepProcess ?stepProcess .
  ?pathwayStep ?relation ?stepProcess .

}
"""

sparql = SPARQLWrapper(endpointURL_reactome_v65)
sparql.setQuery(prefixes+query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/8_processes_of_pathway_steps_linked_to_subpathways.csv", "wb") as f:
    f.write(results)

15:23:51 INFO  Fuseki          :: [15] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.re

pathwayStep,relation,stepProcess
http://www.reactome.org/biopax/65/48887#PathwayStep14662,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12392
http://www.reactome.org/biopax/65/48887#PathwayStep14661,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12391
http://www.reactome.org/biopax/65/48887#PathwayStep14663,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#Catalysis5436
http://www.reactome.org/biopax/65/48887#PathwayStep14663,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12393
http://www.reactome.org/biopax/65/48887#PathwayStep14658,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12389
http://www.reactome.org/biopax/65/48887#PathwayStep14658,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#Catalysis5435
http://www.reactome.org/biopax/65/48887#PathwayStep14659,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12390
http://www.reactome.org/biopax/65/48887#PathwayStep14657,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12388
http://www.reactome.org/biopax/65/48887#PathwayStep14696,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12423
http://www.reactome.org/biopax/65/48887#PathwayStep14686,http://www.biopax.org/release/biopax-level3.owl#stepProcess,http://www.reactome.org/biopax/65/48887#BiochemicalReaction12413


15:23:51 INFO  Fuseki          :: [16] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.re

## Describe the processes linked to pathway steps linked to direct subpathways

In [15]:
defaultPrefixValue = "http://www.reactome.org/biopax/{}/48887#".format(reactomeVersion)
pathwayIdent = "http://www.reactome.org/biopax/65/48887#Pathway2275"
if pathwayIdent.startswith(defaultPrefixValue):
        pathwayIdent = pathwayIdent.replace(defaultPrefixValue, "reactome" + ':')
if not(pathwayIdent.startswith("reactome" + ':')):
        pathwayIdent = "<" + pathwayIdent + ">"
query = """
SELECT DISTINCT ?stepProcess ?properties_of_interest ?description_step_process
WHERE {
  VALUES ?pathwayRoot { """ + pathwayIdent + """ } # Signaling by EGFR
  VALUES ?properties_of_interest { bp3:right bp3:left bp3:participantStoichiometry bp3:controlled bp3:controller bp3:displayName}

  ?pathwayRoot bp3:pathwayComponent ?pathwayCompo .
  ?pathwayCompo bp3:pathwayOrder ?pathwayStep .
  ?pathwayStep bp3:stepProcess ?stepProcess .
  ?stepProcess ?properties_of_interest ?description_step_process .

}
"""

sparql = SPARQLWrapper(endpointURL_reactome_v65)
sparql.setQuery(prefixes+query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)


sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/9_description_processes_pf_ps_linked_to_subpathways.csv", "wb") as f:
    f.write(results)

15:23:51 INFO  Fuseki          :: [17] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.re

stepProcess,properties_of_interest,description_step_process
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12392,http://www.biopax.org/release/biopax-level3.owl#right,http://www.reactome.org/biopax/65/48887#Complex13376
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12391,http://www.biopax.org/release/biopax-level3.owl#right,http://www.reactome.org/biopax/65/48887#Complex2907
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12393,http://www.biopax.org/release/biopax-level3.owl#right,http://www.reactome.org/biopax/65/48887#SmallMolecule158
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12393,http://www.biopax.org/release/biopax-level3.owl#right,http://www.reactome.org/biopax/65/48887#Complex307
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12389,http://www.biopax.org/release/biopax-level3.owl#right,http://www.reactome.org/biopax/65/48887#Complex13375
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12389,http://www.biopax.org/release/biopax-level3.owl#right,http://www.reactome.org/biopax/65/48887#SmallMolecule4
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12390,http://www.biopax.org/release/biopax-level3.owl#right,http://www.reactome.org/biopax/65/48887#Complex4459
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12390,http://www.biopax.org/release/biopax-level3.owl#right,http://www.reactome.org/biopax/65/48887#Protein4393
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12388,http://www.biopax.org/release/biopax-level3.owl#right,http://www.reactome.org/biopax/65/48887#Complex13374
http://www.reactome.org/biopax/65/48887#BiochemicalReaction12423,http://www.biopax.org/release/biopax-level3.owl#right,http://www.reactome.org/biopax/65/48887#Complex13407


15:23:51 INFO  Fuseki          :: [18] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.re

## 10 - Get next steps between pathway steps linked to direct subpathways

In [16]:
defaultPrefixValue = "http://www.reactome.org/biopax/{}/48887#".format(reactomeVersion)
pathwayIdent = "http://www.reactome.org/biopax/65/48887#Pathway2275"
if pathwayIdent.startswith(defaultPrefixValue):
        pathwayIdent = pathwayIdent.replace(defaultPrefixValue, "reactome" + ':')
if not(pathwayIdent.startswith("reactome" + ':')):
        pathwayIdent = "<" + pathwayIdent + ">"
query = """
SELECT DISTINCT ?pathwayStep ?property ?nextStep
WHERE {
  VALUES ?pathwayRoot { """ + pathwayIdent + """ } # Signaling by EGFR
  VALUES ?properties_of_interest { bp3:right bp3:left bp3:participantStoichiometry bp3:controlled bp3:controller bp3:displayName}

  ?pathwayRoot bp3:pathwayComponent ?pathwayCompo .
  ?pathwayCompo bp3:pathwayOrder ?pathwayStep .
  ?pathwayStep bp3:nextStep ?nextStep .
  ?pathwayStep ?property ?nextStep .

}
"""

sparql = SPARQLWrapper(endpointURL_reactome_v65)
sparql.setQuery(prefixes+query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/10_next_steps_between_PS_linked_to_subpathways.csv", "wb") as f:
    f.write(results)

15:23:51 INFO  Fuseki          :: [19] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.re

pathwayStep,property,nextStep
http://www.reactome.org/biopax/65/48887#PathwayStep14662,http://www.biopax.org/release/biopax-level3.owl#nextStep,http://www.reactome.org/biopax/65/48887#PathwayStep16677
http://www.reactome.org/biopax/65/48887#PathwayStep14662,http://www.biopax.org/release/biopax-level3.owl#nextStep,http://www.reactome.org/biopax/65/48887#PathwayStep14663
http://www.reactome.org/biopax/65/48887#PathwayStep14661,http://www.biopax.org/release/biopax-level3.owl#nextStep,http://www.reactome.org/biopax/65/48887#PathwayStep14667
http://www.reactome.org/biopax/65/48887#PathwayStep14661,http://www.biopax.org/release/biopax-level3.owl#nextStep,http://www.reactome.org/biopax/65/48887#PathwayStep15089
http://www.reactome.org/biopax/65/48887#PathwayStep14661,http://www.biopax.org/release/biopax-level3.owl#nextStep,http://www.reactome.org/biopax/65/48887#PathwayStep3294
http://www.reactome.org/biopax/65/48887#PathwayStep14661,http://www.biopax.org/release/biopax-level3.owl#nextStep,http://www.reactome.org/biopax/65/48887#PathwayStep14662
http://www.reactome.org/biopax/65/48887#PathwayStep14661,http://www.biopax.org/release/biopax-level3.owl#nextStep,http://www.reactome.org/biopax/65/48887#PathwayStep15106
http://www.reactome.org/biopax/65/48887#PathwayStep14661,http://www.biopax.org/release/biopax-level3.owl#nextStep,http://www.reactome.org/biopax/65/48887#PathwayStep14913
http://www.reactome.org/biopax/65/48887#PathwayStep14661,http://www.biopax.org/release/biopax-level3.owl#nextStep,http://www.reactome.org/biopax/65/48887#PathwayStep15391
http://www.reactome.org/biopax/65/48887#PathwayStep14658,http://www.biopax.org/release/biopax-level3.owl#nextStep,http://www.reactome.org/biopax/65/48887#PathwayStep14659


15:23:51 INFO  Fuseki          :: [20] GET http://localhost:3030/reactome_v65/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.re

# Merge query results in a single file

In [17]:
q1 = pd.read_csv("../Results/1_direct_pathway_compo.csv", header=None, sep=",")
q1 = q1.drop(q1.index[0]).reset_index(drop=True)
q2 = pd.read_csv("../Results/2_direct_pathway_compo_br.csv", header=None, sep=",")
q2 = q2.drop(q2.index[0]).reset_index(drop=True)
q3 = pd.read_csv("../Results/3_direct_pathway_compo_path.csv", header=None, sep=",")
q3 = q3.drop(q3.index[0]).reset_index(drop=True)
q4 = pd.read_csv("../Results/4_direct_pathway_steps.csv", header=None, sep=",")
q4 = q4.drop(q4.index[0]).reset_index(drop=True)
q5 = pd.read_csv("../Results/5_direct_pathway_steps_next_steps_and_step_process.csv", header=None, sep=",")
q5 = q5.drop(q5.index[0]).reset_index(drop=True)
q6 = pd.read_csv("../Results/6_pathway_steps_steps_processes.csv", header=None, sep=",")
q6 = q6.drop(q6.index[0]).reset_index(drop=True)
q7 = pd.read_csv("../Results/7_linked_to_br_of_direct_pathway.csv", header=None, sep=",")
q7 = q7.drop(q7.index[0]).reset_index(drop=True)
q8 = pd.read_csv("../Results/8_processes_of_pathway_steps_linked_to_subpathways.csv", header=None, sep=",")
q8 = q8.drop(q8.index[0]).reset_index(drop=True)
q9 = pd.read_csv("../Results/9_description_processes_pf_ps_linked_to_subpathways.csv", header=None, sep=",")
q9 = q9.drop(q9.index[0]).reset_index(drop=True)
q10 = pd.read_csv("../Results/10_next_steps_between_PS_linked_to_subpathways.csv", header=None, sep=",")
q10 = q10.drop(q10.index[0]).reset_index(drop=True)


concat_df = pd.concat([q1, q2, q3, q4, q5, q6, q7, q8, q9, q10], ignore_index=True)

concat_df = concat_df.replace({"http://www.reactome.org/biopax/65/48887#": "reactome:", "http://www.biopax.org/release/biopax-level3.owl#": "bp3:"}, regex=True)
print(concat_df)
concat_df.to_csv("../Results/Final_pathway_egf_completed.tsv", sep="\t", header=None, index=False)

                             0                     1  \
0         reactome:Pathway2275  bp3:pathwayComponent   
1         reactome:Pathway2275  bp3:pathwayComponent   
2         reactome:Pathway2275  bp3:pathwayComponent   
3         reactome:Pathway2275  bp3:pathwayComponent   
4         reactome:Pathway2275  bp3:pathwayComponent   
..                         ...                   ...   
933  reactome:PathwayStep14674          bp3:nextStep   
934  reactome:PathwayStep14667          bp3:nextStep   
935  reactome:PathwayStep14667          bp3:nextStep   
936  reactome:PathwayStep14665          bp3:nextStep   
937  reactome:PathwayStep14666          bp3:nextStep   

                                     2  
0    reactome:BiochemicalReaction12383  
1    reactome:BiochemicalReaction12384  
2                 reactome:Pathway2277  
3    reactome:BiochemicalReaction12385  
4                 reactome:Pathway2276  
..                                 ...  
933          reactome:PathwayStep16785  